In [2]:
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [4]:
amazon = quandl.get("WIKI/AMZN")
amazon.tail(7)

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2018-03-19,1554.53,1561.66,1525.35,1544.93,6376619.0,0.0,1.0,1554.53,1561.66,1525.35,1544.93,6376619.0
2018-03-20,1550.34,1587.00,1545.41,1586.51,4507049.0,0.0,1.0,1550.34,1587.00,1545.41,1586.51,4507049.0
2018-03-21,1586.45,1590.00,1563.17,1581.86,4667291.0,0.0,1.0,1586.45,1590.00,1563.17,1581.86,4667291.0
2018-03-22,1565.47,1573.85,1542.40,1544.10,6177737.0,0.0,1.0,1565.47,1573.85,1542.40,1544.10,6177737.0
2018-03-23,1539.01,1549.02,1495.36,1495.56,7843966.0,0.0,1.0,1539.01,1549.02,1495.36,1495.56,7843966.0
2018-03-26,1530.00,1556.99,1499.25,1555.86,5547618.0,0.0,1.0,1530.00,1556.99,1499.25,1555.86,5547618.0
2018-03-27,1572.40,1575.96,1482.32,1497.05,6793279.0,0.0,1.0,1572.40,1575.96,1482.32,1497.05,6793279.0


In [6]:
amazon = amazon[['Adj. Close']]
amazon.tail(7)

,Adj. Close
Date,
2018-03-19,1544.93
2018-03-20,1586.51
2018-03-21,1581.86
2018-03-22,1544.10
2018-03-23,1495.56
2018-03-26,1555.86
2018-03-27,1497.05


In [13]:
forecast_len = 30
amazon['Predicted'] = amazon[['Adj. Close']].shift(- forecast_len)
amazon

,Adj. Close,Predicted
Date,,
1997-05-16,1.729167,1.541667
1997-05-19,1.708333,1.515833
1997-05-20,1.635833,1.588333
1997-05-21,1.427500,1.911667
1997-05-22,1.395833,2.000000
...,...,...
2018-03-21,1581.860000,NaN
2018-03-22,1544.100000,NaN
2018-03-23,1495.560000,NaN


In [15]:
x = np.array(amazon.drop(['Predicted'],1))
x = x[:-forecast_len]
print(x)

[[   1.72916667]
 [   1.70833333]
 [   1.63583333]
 ...
 [1350.47      ]
 [1338.99      ]
 [1386.23      ]]


In [16]:
y = np.array(amazon['Predicted'])
y = y[:-forecast_len]
print(y)

[1.54166667e+00 1.51583333e+00 1.58833333e+00 ... 1.49556000e+03
 1.55586000e+03 1.49705000e+03]


In [17]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [19]:
svr_rbf = SVR(kernel='rbf',C=1e3,gamma=0.1)
svr_rbf.fit(x_train,y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [20]:
svr_rbf_confidence=svr_rbf.score(x_test,y_test)
print(f"SVR Confidence: {round(svr_rbf_confidence*100,2)}%")

SVR Confidence: 92.31%


In [21]:
lr=LinearRegression()
lr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [22]:
lr_confidence=lr.score(x_test,y_test)
print(f"Linear Regression Confidence: {round(lr_confidence*100,2)}%")

Linear Regression Confidence: 98.7%
